In [1]:

#!python3 -m pip install pytorch_lightning
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import LSTM_Model
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import save
from pytorch_lightning.callbacks import EarlyStopping
from torch.utils.data import DataLoader
import neptune.new as neptune
from pytorch_lightning.loggers import NeptuneLogger
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
from BCM_dataset_v2 import bcmDataset, concat_train_test_datasets
import glob
import os
import sys
from collect_dataset import*

Dataset version: 3


In [2]:
loso_datasets = create_loso_datasets()
#indiv_datasets = create_indiv_datasets()
#full_datasets = create_indiv_datasets()
#ft_datasets = create_ft_datasets()


loso_datasets:

train

0
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-34-16
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-52-7
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-15-6
bcm_behaviour_data_multi_subject/subject2/2022-11-09_13-56-0
bcm_behaviour_data_multi_subject/subject3/2022-11-09_15-47-1
bcm_behaviour_data_multi_subject/subject3/2022-11-09_16-26-13
bcm_behaviour_data_multi_subject/subject3/2022-11-09_15-27-14
bcm_behaviour_data_multi_subject/subject3/2022-11-09_16-7-9
1
bcm_behaviour_data_multi_subject/subject3/2022-11-09_15-47-1
bcm_behaviour_data_multi_subject/subject3/2022-11-09_16-26-13
bcm_behaviour_data_multi_subject/subject3/2022-11-09_15-27-14
bcm_behaviour_data_multi_subject/subject3/2022-11-09_16-7-9
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-38-11
bcm_behaviour_data_multi_subject/subject1/2022-09-20_14-58-39
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-18-27
bcm_behaviour_data_multi_subject/subject1/2022-09-2

In [4]:

for i, dataset in enumerate(loso_datasets):
    seed = 1
    seed_everything(seed)
    
    neptune_logger = NeptuneLogger(
        project="NTLAB/BCM-activity-classification",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTA4NzcxMy1lYmQ2LTQ3NTctYjRhNC02Mzk1NjdjMWM0NmYifQ==",
        source_files=["train_model.ipynb", "model.py", "BCM_dataset_v2.py"]
    )

    model = LSTM_Model(dataset, window_size = 3, stride = 1 ,lstm_hidden_size = 128, seed = seed)

    trainer = Trainer(max_epochs=40, 
                    min_epochs=1, 
                    auto_lr_find=False, 
                    auto_scale_batch_size=False,
                    enable_checkpointing=False, 
                    accelerator="gpu", 
                    devices = 1, 
                    logger=neptune_logger)
    trainer.tune(model)
    trainer.fit(model)
    
    model_scripted = torch.jit.script(model) # Export to TorchScript
    model_scripted.save(f"trained_models/loso_model_val_subject_{i}") # Save
    
    # Stop logging
    neptune_logger.experiment.stop()

Global seed set to 1


_____________________
<class 'list'>
_____________________
Validation set
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-38-11/0.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-38-11/1.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-38-11/2.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-38-11/3.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-38-11/4.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_14-58-39/0.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_14-58-39/1.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_14-58-39/2.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_14-58-39/3.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_14-58-39/4.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-18-27/0.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-18-27/1.npy
bcm_behaviour_data_multi_subject/subject1/2022-09-20_15-18-27/2.npy
bcm_behaviour_data_multi_subject/subject1/

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


https://app.neptune.ai/NTLAB/BCM-activity-classification/e/BCMAC-668
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.



  | Name         | Type             | Params
--------------------------------------------------
0 | lstm         | LSTM             | 149 K 
1 | flatten      | Flatten          | 0     
2 | fc           | Linear           | 1.3 K 
3 | output       | Sigmoid          | 0     
4 | sm           | Softmax          | 0     
5 | loss         | CrossEntropyLoss | 0     
6 | accuracy     | Accuracy         | 0     
7 | val_accuracy | Accuracy         | 0     
--------------------------------------------------
150 K     Trainable params
0         Non-trainable params
150 K     Total params
0.603     Total estimated model params size (MB)


...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=40` reached.


{'progress_bar': {'val_loss': 1.4492766857147217}, 'log': {'val_loss': 1.4492766857147217}, 'val_loss': 1.4492766857147217}
Accuracy: 0.8465180993080139
Accuracy: 0.9362117052078247
Shutting down background jobs, please wait a moment...
Done!


/home/aron/.local/lib/python3.9/site-packages/torch/jit/_recursive.py:621: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(mod, name, None)
/home/aron/.local/lib/python3.9/site-packages/torch/jit/_recursive.py:708: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(nn_module, name, None)
/home/aron/.local/lib/python3.9/site-packages/torch/jit/_recursive.py:508: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(nn_module, name, None)
/home/aron/.local/lib/python3.9/site-packages/torch/jit/_recursive.py:591: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(nn_mo

All 57 operations synced, thanks for waiting!


In [ ]:


#model_scripted = torch.jit.script(model) # Export to TorchScript
#model_scripted.save("trained_models/oct_31") # Save